Importando as bibliotecas

In [1]:
import numpy as np
import pandas as pd
import sweetviz as sv
import os
import warnings
warnings.filterwarnings("ignore")

Garantindo que o diretório de trabalho está correto

In [2]:
base_dir = os.path.dirname(os.path.abspath('__file__'))
dir_files_path = os.path.join(base_dir, '../data/raw/2016.csv')
reports_file_path = os.path.join(base_dir, '../reports/2016')

Dataframes

Após ler o dicionário de dados, percebi que essas 3 bases são as mais relevantes para o estudo.

In [3]:
licitacoes_2016 = pd.read_csv(f"{dir_files_path}/licitacao.csv")
lotes_2016 = pd.read_csv(f"{dir_files_path}/lote.csv")
itens_2016 = pd.read_csv(f"{dir_files_path}/item.csv")

Relatórios de análise exploratória

In [ ]:
# licitacoes
report_licitacoes = sv.analyze(licitacoes_2016)
report_licitacoes.show_html(filepath=f"{reports_file_path}/licitacoes_report.html", layout='vertical')

In [ ]:
#itens
report_itens = sv.analyze(itens_2016)
report_itens.show_html(filepath=f"{reports_file_path}/itens_report.html", layout= 'vertical')

O erro anterior aponta que na coluna VL_TOTAL_HOMOLOGADO consta uma combinação de tipos de variáveis - float e string
Logo, precisa-se explorar o dataframe para identificar o que está acontecendo

In [ ]:
itens_2016.shape # (456368, 32)

In [ ]:
itens_2016.info()

A coluna VL_TOTAL_HOMOLOGADO não consta no dicionário de dados apresentado e nem no site da transparência.
Irei tratar o dado para verificar o report, mas acredito que não seja um dado relevante para o estudo.

In [ ]:
itens_2016[['QT_ITENS','VL_UNITARIO_HOMOLOGADO', 'VL_TOTAL_HOMOLOGADO']]

In [ ]:
itens_2016[['VL_UNITARIO_HOMOLOGADO','VL_TOTAL_HOMOLOGADO']].isnull().sum()

In [ ]:
372973 + 83395
# A soma dos valores nulos e não nulos é igual ao número de linhas do dataset

Então, vou tratar os valores nulos e qualquer outro problema que possa existir para gerar o report

In [ ]:
# Tratando possíveis problemas com os valores em reais
itens_2016['VL_TOTAL_HOMOLOGADO'] = itens_2016['VL_TOTAL_HOMOLOGADO'].str.replace(',','.')
itens_2016['VL_TOTAL_HOMOLOGADO'] = itens_2016['VL_TOTAL_HOMOLOGADO'].str.replace('R$','')

In [ ]:
# Forçando a conversão para float
itens_2016['VL_TOTAL_HOMOLOGADO'] = pd.to_numeric(itens_2016['VL_TOTAL_HOMOLOGADO'], errors='coerce')

In [ ]:
# Preenchendo os valores nulos com 0
itens_2016[['VL_UNITARIO_HOMOLOGADO','VL_TOTAL_HOMOLOGADO']] = itens_2016[['VL_UNITARIO_HOMOLOGADO','VL_TOTAL_HOMOLOGADO']].fillna(0)

In [ ]:
# Checando se ainda há valores nulos
itens_2016[['VL_UNITARIO_HOMOLOGADO','VL_TOTAL_HOMOLOGADO']].isnull().sum()

Agora, consigo converter os valores para float

In [ ]:
# this converts the values to float
itens_2016['VL_TOTAL_HOMOLOGADO'] = itens_2016['VL_TOTAL_HOMOLOGADO'].astype(float)

E, finalmente, gerar o report

In [ ]:
#itens
report_itens = sv.analyze(itens_2016)
report_itens.show_html(filepath=f"{reports_file_path}/itens_report.html", layout= 'widescreen')

Levando em conta que a coluna apresentada no erro anterior também não está no dicionário de dados, vou ignorá-la em conjunto com qualquer outra para gerar o report

In [ ]:
#itens
report_itens = sv.analyze(itens_2016.drop(columns=['NR_DOCUMENTO.1']))
report_itens.show_html(filepath=f"{reports_file_path}/itens_report.html", layout= 'vertical')

Por meio dos reports, é possível fazer um diagnóstico inicial do dataset e identificar possíveis respostas para as perguntas de negócio.

Agora, iremos começar pelas bases recomendadas e depois tentar relacionar com as demais com o objetivo de adquirir mais informações
e por fim responder às perguntas de negócio.

In [4]:
# licitacoes filtradas pelo tipo de objeto - compra
filtro = licitacoes_2016['TP_OBJETO'].str.match('com',case = False)
licitacoes_2016_compras = licitacoes_2016[filtro]
licitacoes_2016_compras ['DS_OBJETO'].value_counts()

...                                                                                                                                                           176
Material                                                                                                                                                       69
AQUISIÇÃO DE MEDICAMENTOS                                                                                                                                      36
Aquisição de medicamentos                                                                                                                                      24
Aquisição de material hospitalar                                                                                                                               23
                                                                                                                                                             ... 
AQUISIÇÃO DE MOTOCICLETA, ZE

É possível observar que a maioria das descrições de liciatação contém a palavra "aquisição" e sinonimos para se referir a compra de bens. Logo, desenvolvi uma função utilizando NLTK para identificar os substantivos e, assim, identificar maiores relações entre os dados.

In [5]:
from my_library import get_most_common_nouns

get_most_common_nouns(licitacoes_2016_compras,'DS_OBJETO', 100)

[nltk_data] Downloading package punkt to /home/vlgi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/vlgi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[('DE', 11416),
 ('Aquisição', 4753),
 ('PARA', 4309),
 ('AQUISIÇÃO', 3562),
 ('E', 3032),
 ('DA', 2008),
 ('DO', 1736),
 ('A', 1200),
 ('MUNICIPAL', 1043),
 ('Municipal', 986),
 ('MATERIAL', 904),
 ('Secretaria', 900),
 ('SECRETARIA', 852),
 ('REGISTRO', 725),
 ('O', 690),
 ('Registro', 685),
 ('Saúde', 680),
 ('MATERIAIS', 671),
 ('PREÇOS', 613),
 ('Nº', 544),
 ('CONFORME', 528),
 ('EQUIPAMENTOS', 511),
 ('SAÚDE', 489),
 ('Município', 488),
 ('MANUTENÇÃO', 460),
 ('MUNICÍPIO', 451),
 ('COM', 419),
 ('AS', 394),
 ('Preços', 365),
 ('DAS', 360),
 ('ESCOLAR', 328),
 ('NA', 323),
 ('EM', 322),
 ('AQUISICAO', 306),
 ('ALIMENTÍCIOS', 303),
 ('GÊNEROS', 302),
 ('NO', 301),
 ('USO', 297),
 ('Contratação', 294),
 ('PREGÃO', 291),
 ('MUNICIPAIS', 272),
 ('Educação', 270),
 ('MEDICAMENTOS', 268),
 ('Fornecimento', 267),
 ('DIVERSOS', 265),
 ('ATRAVÉS', 261),
 ('CONTRATAÇÃO', 257),
 ('Agricultura', 256),
 ('ESCOLAS', 253),
 ('EMPRESA', 250),
 ('EDUCAÇÃO', 246),
 ('Material', 241),
 ('Social', 23

In [6]:
licitacoes_2016_compras['NM_ORGAO'].value_counts()

FUND. DE SAÚDE PÚBLICA DE NOVO HAMBURGO                     244
PM DE VIAMÃO                                                244
PM DE PELOTAS                                               238
PM DE GUAPORÉ                                               238
PM DE NOVO HAMBURGO                                         190
                                                           ... 
INST. DE PREV. E ASSIST. DOS SERV. MUNIC. DE N. HAMBURGO      1
PREV-XANGRI-LÁ                                                1
INST. MUN. ASSIST. SERV. DE NOVA SANTA RITA                   1
CM DE CONSTANTINA                                             1
CM DE SÃO PEDRO DA SERRA                                      1
Name: NM_ORGAO, Length: 577, dtype: int64

In [7]:
licitacoes_2016_compras.groupby(['CD_ORGAO','NM_ORGAO'])['VL_LICITACAO'].sum().sort_values(ascending=False)

CD_ORGAO  NM_ORGAO                                     
72600     PM DE TUPANDI                                    7.920721e+08
88258     CONS. PÚBL. DO EXTREMO SUL                       4.934806e+08
63000     PM DE VIAMÃO                                     4.207760e+08
44500     PM DE CANOAS                                     2.162334e+08
88190     CONS. INTERM. DE SAÚDE DO VALE DO RIO TAQUARI    8.848445e+07
                                                               ...     
60500     PM DE SERTÃO                                     0.000000e+00
82701     CM DE XANGRI-LÁ                                  0.000000e+00
61500     PM DE TORRES                                     0.000000e+00
68400     PM DE IPIRANGA DO SUL                            0.000000e+00
60600     PM DE SEVERIANO DE ALMEIDA                       0.000000e+00
Name: VL_LICITACAO, Length: 577, dtype: float64

In [8]:
itens_2016

,CD_ORGAO,NR_LICITACAO,ANO_LICITACAO,CD_TIPO_MODALIDADE,NR_LOTE,NR_ITEM,NR_ITEM_ORIGINAL,DS_ITEM,QT_ITENS,SG_UNIDADE_MEDIDA,...,TP_ORCAMENTO,CD_TIPO_FAMILIA,CD_TIPO_SUBFAMILIA,TP_DOCUMENTO,NR_DOCUMENTO,TP_DOCUMENTO.1,NR_DOCUMENTO.1,TP_BENEFICIO_MICRO_EPP,PC_TX_ESTIMADA,PC_TX_HOMOLOGADA
0,77200,15.0,2016,PRP,1,2,2,"Pneu 1000/20 liso radial, dianteiro, 16 lonas,...",10.0,UN,...,M,745.0,632.0,J,8.774832e+12,NaN,NaN,L,NaN,NaN
1,77200,15.0,2016,PRP,1,4,4,"Pneu 12.4x24 garra baixa, 10 lonas",4.0,UN,...,M,745.0,632.0,J,8.774832e+12,NaN,NaN,L,NaN,NaN
2,77200,15.0,2016,PRP,1,7,7,Pneu 1400x24 mínimo 12 lonas,6.0,UN,...,M,745.0,632.0,J,8.774832e+12,NaN,NaN,L,NaN,NaN
3,77200,15.0,2016,PRP,1,8,8,"Pneu 17.5x25 12 lonas, capacidade mínima de ca...",4.0,UN,...,M,745.0,632.0,J,8.774832e+12,NaN,NaN,L,NaN,NaN
4,77200,15.0,2016,PRP,1,12,12,"Pneu 18.4x30 garra baixa, mínimo 10 lonas",4.0,UN,...,M,745.0,632.0,J,8.774832e+12,NaN,NaN,L,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456363,78900,13.0,2016,PRP,1,154,154,Ranitidina 150mg,10000.0,COM,...,NaN,NaN,NaN,J,5.782733e+12,NaN,NaN,N,NaN,NaN
456364,78900,13.0,2016,PRP,1,156,156,Salbutamol 100mcg/dose aerosol 200 doses,300.0,FR,...,NaN,NaN,NaN,J,5.782733e+12,NaN,NaN,N,NaN,NaN
456365,78900,13.0,2016,PRP,1,158,158,Sinvastatina 40mg,100000.0,COM,...,NaN,NaN,NaN,J,5.782733e+12,NaN,NaN,N,NaN,NaN
456366,78900,13.0,2016,PRP,1,167,167,Valproato de Sódio 500 mg,15000.0,COM,...,NaN,NaN,NaN,J,5.782733e+12,NaN,NaN,N,NaN,NaN


In [9]:
itens_2016['DS_ITEM'].value_counts()

MANGUEIRA SILICONE 2.1/2"              1360
MATERIAL                                873
LIVRO                                   355
.                                       344
NÃO CADASTRADO                          304
                                       ... 
Resina fotopolimerizavel , cor A3,5       1
Resina fotopolimerizavel cor:A3           1
Resina fotopolimerizavel, cor B2          1
Resina fotopolimerizável cor A2           1
Valproato de Sódio 500 mg                 1
Name: DS_ITEM, Length: 359635, dtype: int64

Por meio do resultado anterior é possível determinar algumas estratégias para a exploração de dados sobre os itens licitados.
Como por exemplo:
- Descrições que possuam a string 'mg'/'miligramas' ou 'ml'/'mililitros' podem ser relacionadas com medicamentos
- Descrições que possuam unidades de medida como m², m³, etc podem ser relacionadas com materiais de construção
- Descrições que possuam a string 'kg'/'quilogramas' podem ser relacionadas com alimentos

In [10]:
itens_2016[itens_2016['DS_ITEM'].str.contains('mg', case = False)]['DS_ITEM'].value_counts()

FUROSEMIDA 40MG                                42
CARBAMAZEPINA 200MG                            41
FENOBARBITAL 100MG                             40
PREDNISONA 5MG                                 38
CEFALEXINA 500MG                               38
                                               ..
DIPIRONA 500 MG/ML SOLUÇÃO ORAL 10 ML           1
ENALAPRIL 20MG (EMBALAGEM HOSPITALAR)           1
ESPIRONOLACTONA 50MG (EMBALAGEM HOSPITALAR)     1
FENOBARBITAL 100 MG (EMBALAGEM HOSPITALAR)      1
Valproato de Sódio 500 mg                       1
Name: DS_ITEM, Length: 27872, dtype: int64

In [11]:
itens_2016[itens_2016['DS_ITEM'].str.contains('ml', case = False)]['DS_ITEM'].value_counts()

ÓLEO DE SOJA 900 ML                                                                                                                  21
OLEO DE SOJA 900 ML                                                                                                                  20
ÓLEO DE SOJA VEGETAL: garrafa de 900ml                                                                                               15
VINAGRE DE MAÇÃ: garrafa de 750ml                                                                                                    13
OLEO DE SOJA 900ML                                                                                                                   11
                                                                                                                                     ..
Cola gliter, não tóxica, composta por resinas de pva, gliter e conservante tipo benzoatiazol, embalagem de 35 ml, na cor prata        1
Cola gliter, não tóxica, composta por resinas de